# Game of Zones: Using Machine Learning to Predict the Outcome of an MLB At Bat

Goal: predict where the batter is most likely to hit the ball (zones of the field) in an at-bat given the situation and the pitcher he is facing
    
Input Data: 
- the pitcher's repertoire: given that each pitcher has a different arsenal of pitches and each pitch moves differently, we use a cluster analysis to categorize pitch types.  In this way, we put each pitcher on the same footing.

- the game situation: the inning (and top/bottom), the number of outs, positions of baserunners, the count, positions of fielders(?)

- the batter's priors: distribution of batted balls into zones

Output: 
- probabilities for each zone on the field where the batter can hit the ball

- contributing factors for each prediction (things the defensive team could use to intervene)

In [32]:
%load_ext autoreload
%autoreload 2
from pybaseball import statcast
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from matplotlib import patches
%matplotlib inline

from get_data import get_data, get_hit_zone, get_pitch_data
from pitch_clustering import pitch_clustering

# use Statcast data (from 2015-2018) so we can get spin rate, etc.
train_data_dates = [('2015-04-05', '2015-10-04')] #,
#                     ('2016-04-03', '2016-10-02'),
#                     ('2017-04-02', '2017-10-01'),
#                     ('2018-03-29', '2018-10-01')]

## Build the Outcome

In [3]:
# get the outcome data
outcome_data = get_data(get_hit_zone, train_data_dates)

# write to file
outcome_data.to_csv("./outcome_4zones.csv", index=False)

print(f"Shape of the outcome data: {outcome_data.shape}")
outcome_data.head()

This is a large query, it may take a moment to complete
Completed sub-query from 2015-04-05 to 2015-04-10
Completed sub-query from 2015-04-11 to 2015-04-16
Completed sub-query from 2015-04-17 to 2015-04-22
Completed sub-query from 2015-04-23 to 2015-04-28
Completed sub-query from 2015-04-29 to 2015-05-04
Completed sub-query from 2015-05-05 to 2015-05-10
Completed sub-query from 2015-05-11 to 2015-05-16
Completed sub-query from 2015-05-17 to 2015-05-22
Completed sub-query from 2015-05-23 to 2015-05-28
Completed sub-query from 2015-05-29 to 2015-06-03
Completed sub-query from 2015-06-04 to 2015-06-09
Completed sub-query from 2015-06-10 to 2015-06-15
Completed sub-query from 2015-06-16 to 2015-06-21
Completed sub-query from 2015-06-22 to 2015-06-27
Completed sub-query from 2015-06-28 to 2015-07-03
Completed sub-query from 2015-07-04 to 2015-07-09
Completed sub-query from 2015-07-10 to 2015-07-15
Completed sub-query from 2015-07-16 to 2015-07-21
Completed sub-query from 2015-07-22 to 2015-

,game_pk,index,batter,pitcher,hit_zone
0,416079,239,150029,544727,4
2,416079,264,547180,544727,2
10,416079,330,543685,544727,1
15,416079,395,502517,595014,2
24,416079,529,434158,595014,4


## Build the Pitcher Data

### Get the Pitch Data from Baseball Savant

In [6]:
# get the outcome data
pitch_data = get_data(get_pitch_data, train_data_dates)

# write to file
pitch_data.to_csv("./pitch.csv", index=False)

# print the number of pitchers in the data set
print(f"Number of pitchers in the data: {len(pitch_data['pitcher'].unique())}")

# drop rows with missing values
pitch_data.dropna(inplace=True)
print(f"Shape of training data: {pitch_data.shape}")
      
# extract the pitcherID column
pitcherID = pitch_data[['pitcher', 'player_name']]
pitch_data = pitch_data.drop(['pitcher', 'player_name'], axis=1)

pitch_data.head()

This is a large query, it may take a moment to complete
Completed sub-query from 2015-04-05 to 2015-04-10
Completed sub-query from 2015-04-11 to 2015-04-16
Completed sub-query from 2015-04-17 to 2015-04-22
Completed sub-query from 2015-04-23 to 2015-04-28
Completed sub-query from 2015-04-29 to 2015-05-04
Completed sub-query from 2015-05-05 to 2015-05-10
Completed sub-query from 2015-05-11 to 2015-05-16
Completed sub-query from 2015-05-17 to 2015-05-22
Completed sub-query from 2015-05-23 to 2015-05-28
Completed sub-query from 2015-05-29 to 2015-06-03
Completed sub-query from 2015-06-04 to 2015-06-09
Completed sub-query from 2015-06-10 to 2015-06-15
Completed sub-query from 2015-06-16 to 2015-06-21
Completed sub-query from 2015-06-22 to 2015-06-27
Completed sub-query from 2015-06-28 to 2015-07-03
Completed sub-query from 2015-07-04 to 2015-07-09
Completed sub-query from 2015-07-10 to 2015-07-15
Completed sub-query from 2015-07-16 to 2015-07-21
Completed sub-query from 2015-07-22 to 2015-

,pitcher,player_name,release_speed,release_spin_rate,release_extension,pfx_x,pfx_z,plate_x,plate_z,vx0,vy0,vz0,ax,ay,az
0,544727.0,Jeurys Familia,97.2,2018.0,6.125,-1.118083,0.455133,-0.707,2.022,3.602,-141.273,-6.540,-14.347,32.145,-28.506
1,544727.0,Jeurys Familia,97.5,2093.0,6.030,-1.108342,0.711700,0.442,3.496,6.394,-141.716,-3.746,-14.306,32.100,-24.835
2,544727.0,Jeurys Familia,98.4,1960.0,6.359,-1.523058,0.523933,-0.353,2.446,5.481,-143.007,-5.675,-20.489,36.599,-27.465
3,544727.0,Jeurys Familia,97.7,2099.0,6.275,-1.285083,0.618533,-0.054,1.273,5.577,-141.819,-8.799,-16.942,31.897,-26.150
4,544727.0,Jeurys Familia,98.2,2155.0,6.538,-0.997008,0.803433,-0.406,1.560,3.987,-142.748,-8.509,-12.709,36.274,-23.516


In [ ]:
pitch_clustering(pitch_data)

Number of components from PCA: 7


### Rescale the Data

In [20]:
scaler = MinMaxScaler()
train_data_scaled = pd.DataFrame(scaler.fit_transform(train_data))
train_data_scaled.columns = train_data.columns

train_data_scaled.head()

,release_speed,release_spin_rate,release_extension,pfx_x,pfx_z,plate_x,plate_z,vx0,vy0,vz0,ax,ay,az
0,0.898529,0.506169,0.547715,0.336825,0.456100,0.516871,0.392762,0.577133,0.101997,0.331849,0.281855,0.620331,0.410816
1,0.902941,0.531177,0.539428,0.338419,0.496921,0.577726,0.494795,0.643021,0.097651,0.419270,0.282395,0.619328,0.469035
2,0.916176,0.486829,0.568126,0.270553,0.467047,0.535620,0.422112,0.621475,0.084986,0.358914,0.200964,0.719600,0.427325
3,0.905882,0.533178,0.560799,0.309497,0.482098,0.551456,0.340915,0.623740,0.096640,0.261168,0.247679,0.614803,0.448180
4,0.913235,0.551851,0.583740,0.356639,0.511517,0.532813,0.360782,0.586218,0.087527,0.270242,0.303428,0.712356,0.489953


### Use PCA to Reduce Dimensionality of Data

In [ ]:
#Fitting the PCA algorithm with our Data
pca = PCA().fit(train_data_scaled)

#Plotting the Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Pitch Dataset Explained Variance')
plt.show()

#### Fit/Transform using the number of components that get us to 98% variance explained

In [ ]:
pca = PCA(n_components=7)
train_data_pca = pd.DataFrame(pca.fit_transform(train_data_scaled))
train_data_pca.head()

### K-Means Clustering of the Pitch Data

#### use the Elbow method to find optimal k

In [ ]:
# range of number of clusters
num_clusters = range(3, 16)

# initiate the models
kmeans = []
for i in num_clusters:
    print(f"initializing a model for {i} clusters")
    kmeans.append(KMeans(n_clusters=i, n_jobs=-1))

# compute the scores for each fit
score = []
for i, k in enumerate(num_clusters):
    print(f"fitting the data to {k} clusters")
    score.append(kmeans[i].fit(train_data_pca).score(train_data_pca))

# # cluster labels
# cluster_labels = []
# for i, k in enumerate(num_clusters):
#     print(f"labeling the data for {k} clusters")
#     cluster_labels.append(kmeans[i].fit_predict(train_data_pca))

In [ ]:
plt.plot(num_clusters, score)
plt.title("Elbow Curve")
plt.xlabel("Number of Clusters")
plt.ylabel("K-Means Score")
plt.show()

#### k-means with Gap statistic to find optimal k

In [ ]:
def optimalK(data, nrefs=3, maxClusters=15):
    """
    Calculates KMeans optimal K using Gap Statistic from Tibshirani, Walther, Hastie
    Params:
        data: ndarry of shape (n_samples, n_features)
        nrefs: number of sample reference datasets to create
        maxClusters: Maximum number of clusters to test for
    Returns: (gaps, optimalK)
    """
    gaps = np.zeros((len(range(1, maxClusters)),))
    resultsdf = pd.DataFrame({'clusterCount':[], 'gap':[]})
    for gap_index, k in enumerate(range(1, maxClusters)):

        # Holder for reference dispersion results
        refDisps = np.zeros(nrefs)

        # For n references, generate random sample and perform kmeans getting resulting dispersion of each loop
        for i in range(nrefs):
            
            # Create new random reference set
            randomReference = np.random.random_sample(size=data.shape)
            
            # Fit to it
            km = KMeans(k, n_jobs=-1)
            km.fit(randomReference)
            
            refDisp = km.inertia_
            refDisps[i] = refDisp

        # Fit cluster to original data and create dispersion
        km = KMeans(k)
        km.fit(data)
        
        origDisp = km.inertia_

        # Calculate gap statistic
        gap = np.log(np.mean(refDisps)) - np.log(origDisp)

        # Assign this loop's gap statistic to gaps
        gaps[gap_index] = gap
        
        resultsdf = resultsdf.append({'clusterCount':k, 'gap':gap}, ignore_index=True)

    return (gaps.argmax() + 1, resultsdf)  # Plus 1 because index of 0 means 1 cluster is optimal, index 2 = 3 clusters are optimal

k, _ = optimalK(train_data_pca)
print(f"The optimal number of clusters is: {k}")

#### train k-means model with optimal number of clusters

In [ ]:
kmeans = KMeans(n_clusters=k, n_jobs=-1)
kmeans.fit(train_data_pca)

# build a dataframe to contain the pitch classifications
labels = pd.DataFrame(kmeans.labels_)
labels.head()

#### turn the labels into counts of pitch type for each pitcher 

In [ ]:
# first, one-hot-encode the label df
ohe = OneHotEncoder()
labels_ohe = pd.DataFrame(ohe.fit_transform(labels).toarray())

# give the new columns names
pitch_labels = ['pitch_type_'+str(i) for i in range(len(labels_ohe.columns.tolist()))]
labels_ohe.columns = pitch_labels

# merge the pitch type data with the pitcher's ID and name
pitch_data = pd.merge(pitcherID, labels_ohe, left_index=True, right_index=True)

# groupby on the pitcher ID/name and sum the rows
pitch_data = pitch_data.groupby(['pitcher', 'player_name']).sum()
pitch_data.reset_index(inplace=True, drop=False)

# finally, turn the pitch counts into percentages
pitch_data_pct = pd.DataFrame()
for i in range(len(pitch_data)):
    temp_df = pd.DataFrame(pitch_data.iloc[i]).T
    pitch_sum = sum(temp_df.iloc[0,2:])
    temp_df.iloc[0,2:] = temp_df.iloc[0,2:] / pitch_sum
    pitch_data_pct = pitch_data_pct.append(temp_df)
    
data_file_path = "/home/chris/data/baseball/PitchClustering/"
pitch_data_pct.to_csv(data_file_path + "pitch_11types.csv", index=False)
    
print(pitch_data_pct.shape)
pitch_data_pct.head(20)

## Build the Batter's Prior Zone Distribution

In [ ]:
# use the hit zone data from the outcome calculated above
data_file_path = "/home/chris/data/baseball/HitZone/"
batter_zone_data = pd.read_csv(data_file_path + "train_4zones.csv")
batter_zone_data = batter_zone_data[['batter', 'hit_zone']]
batter_zone_data.set_index('batter', inplace=True)

# one-hot-encode the hit_zone column
ohe = OneHotEncoder()
zones_ohe = pd.DataFrame(ohe.fit_transform(batter_zone_data).toarray())
zones_ohe.index = batter_zone_data.index
zones_ohe.columns = ['zone_1', 'zone_2', 'zone_3', 'zone_4']
zones_ohe.reset_index(inplace=True, drop=False)

# group by batter ID and sum
batter_zone_data = zones_ohe.groupby('batter').sum()

# finally, turn the pitch counts into percentages
batter_zone_data_pct = pd.DataFrame()
for i in range(len(batter_zone_data)):
    temp_df = pd.DataFrame(batter_zone_data.iloc[i]).T
    pitch_sum = sum(temp_df.iloc[0,:])
    temp_df.iloc[0,:] = temp_df.iloc[0,:] / pitch_sum
    batter_zone_data_pct = batter_zone_data_pct.append(temp_df)
    
batter_zone_data_pct.reset_index(inplace=True, drop=False)
batter_zone_data_pct.columns = ['batter', 'batter_zone_1', 'batter_zone_2', 'batter_zone_3', 'batter_zone_4']

batter_zone_data_pct.to_csv("/home/chris/data/baseball/HitZone/batter_prior_4zones.csv", index=False)

print(batter_zone_data_pct.shape)
batter_zone_data_pct.head()

In [ ]:
batter_zone_data_pct.describe()

## Build the Complete Feature Set for Training the Model

### Get the Game Situation Features

In [ ]:
def get_features(start, end):

    data = statcast(start_dt=start, end_dt=end)

    # just keep the "events" (i.e., the end result of an at-bat)
    data = data[~pd.isnull(data['events'])]

    # remove events that don't involve the ball being put into play in a meaningful way 
    # (i.e., no strikeouts, walks, sacrifice bunts, caught stealing, hit by pitch, etc.)
    inplay_event_list = ['field_out', 'sac_fly', 'force_out', 'field_error', 'double', 'home_run', 
                         'grounded_into_double_play', 'fielders_choice', 'fielders_choice_out', 'triple', 
                         'double_play']
    data = data[data['events'].isin(inplay_event_list)]

    # trim down the features 
    cols_to_keep = ['game_pk', 'index', 'batter', 'pitcher', 'stand', 'p_throws', 'balls', 'strikes', 'outs_when_up', 
                    'inning', 'on_1b', 'on_2b', 'on_3b', 'bat_score', 'fld_score']
    data = data[cols_to_keep]

    # make sure index columns are int's
    for col in ['game_pk', 'index', 'batter', 'pitcher']:
        data[col] = data[col].astype(int)

    data['bat_right'] = data['stand'].apply(lambda x: x == 'R') 
    data['pitch_right'] = data['p_throws'].apply(lambda x: x == 'R')
    data.drop(['stand', 'p_throws'], axis=1, inplace=True)

    for col in ['on_1b', 'on_2b', 'on_3b']:
        data[col] = data[col].apply(lambda x: x == x)

    data['score_diff'] = data['bat_score'] - data['fld_score']
    data.drop(['bat_score', 'fld_score'], axis=1, inplace=True)
    
    return data

In [ ]:
# use Statcast data (from 2015-2018) so we can get spin rate, etc.
train_data_dates = [('2015-04-05', '2015-10-04'),
                    ('2016-04-03', '2016-10-02'),
                    ('2017-04-02', '2017-10-01'),
                    ('2018-03-29', '2018-10-01')]

# build a list of dataframes (one for each year)
train_data_list = []
for dates in train_data_dates:
    df = get_features(start=dates[0], end=dates[1])
    train_data_list.append(df)
    
# concat the list of dataframes into one large dataframe
train_data = pd.concat(train_data_list)

# write to file
data_file_path = "/home/chris/data/baseball/HitFeatures/"
train_data.to_csv(data_file_path + "train.csv", index=False)

print(train_data.shape)
train_data.head()

### Combine the Game Situation, Pitcher and Batter Features along with the Outcome

In [ ]:
game_situation_df = pd.read_csv("/home/chris/data/baseball/HitFeatures/train.csv")

pitch_type_df = pd.read_csv("/home/chris/data/baseball/PitchClustering/pitch_11types.csv")

pitch_type_df.drop('player_name', axis=1, inplace=True)
pitch_type_df['pitcher'] = pitch_type_df['pitcher'].astype(int)

batter_zone_df = pd.read_csv("/home/chris/data/baseball/HitZone/batter_prior_4zones.csv")

outcome_df = pd.read_csv("/home/chris/data/baseball/HitZone/train_4zones.csv")

full_data = pd.merge(game_situation_df, pitch_type_df, on="pitcher")
full_data = pd.merge(full_data, batter_zone_df, on="batter")
full_data = pd.merge(outcome_df, full_data, on=['game_pk', 'index', 'batter', 'pitcher'])

print(game_situation_df.shape)
print(pitch_type_df.shape)
print(batter_zone_df.shape)
print(outcome_df.shape)
print(full_data.shape)
full_data.head()

In [ ]:
full_data = full_data.drop(['game_pk', 'index', 'batter', 'pitcher'], axis=1)

# split the dataframe into a feature set and an outcome column
X = full_data.drop('hit_zone', axis=1)
y = full_data['hit_zone']

# split the data into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4256)

In [ ]:
# ----------------------
# train an XGBoost model
# ----------------------

# small set of hyperparameters to optimize over
xgb_params = {"max_depth": (3, 5, 10, 15, 20),
              "learning_rate": (0.01, 0.5, 0.1, 0.2, 0.4),
              "gamma": (0, 33, 66, 100),
              "min_child_weight": (0, 33, 66, 100),
              "colsample_bytree": (0.5, 0.75, 1),
              "subsample": (0.5, 0.75, 1),}

# perform the paramater grid search using 5-fold cross validation
xgb_opt = GridSearchCV(XGBClassifier(objective='multi:softprob', num_class=4), 
                       param_grid=xgb_params, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

#xgb_opt = XGBClassifier(objective='multi:softprob', num_class=4)

# perform fit and make predictions
xgb_opt.fit(X_train, y_train)
y_pred = xgb_opt.predict(X_test)
y_prob = xgb_opt.predict_proba(X_test)

# compute accuracy
accuracy = round(accuracy_score(y_test, y_pred) * 100, 1)

# print the confusion matrix
print(confusion_matrix(y_test, y_pred))

In [ ]:
accuracy

In [ ]:
features = X_train.columns.tolist()
importances = list(xgb_opt.feature_importances_)
for i in range(len(features)):
    print(features[i] + "\t" + str(importances[i] * 100.))